### Sales And Marketing Report

In [141]:
# pip install xlrd

In [142]:
import unicodedata
import os
import sys
import re
import numpy as np
import pandas as pd
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof

##### Functions

In [143]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [144]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [145]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [146]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [147]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [218]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [219]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text = text.strip()  # rstrip lstrip
    return text

#### Fiyat Listesi

In [220]:
df_price = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Satış Pazarlama\Fiyat Listesi\KATEGORİ FİYAT LİSTESİ.xlsx")
df_price

,Ürün Adı,Model Kodu,Gelişi (KDV DAHİL)
0,T2010,004166,1242.0
1,T2011,004207,702.0
2,T2013,004206,864.0
3,T2014,004167,972.0
4,EV17,003138,972.0
...,...,...,...
317,buhurdanlık yaşam ağacı,002906,35.0
318,buhurdanlık lotus,002906,35.0
319,buhurdanlık yaşam çiçeği,002906,35.0
320,buhurdanlık yaprak,002906,35.0


In [221]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ürün Adı            322 non-null    object 
 1   Model Kodu          322 non-null    object 
 2   Gelişi (KDV DAHİL)  322 non-null    float64
dtypes: float64(1), object(2)
memory usage: 7.7+ KB


#### N11

In [222]:
df_n11_raw = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Satış Pazarlama\N11 SATIŞ RAPORLARI\haziran 2024 n11...xls")
df_n11_raw

,Sipariş Kodu,Durum,Sipariş Tamamlanma Tarihi,Ödeme Tarihi,Ürün Kodu,Mağaza Ürün Kodu,Stok Kodu,Ürün Adı,Seçenek,Birim Fiyatı,...,Vergi Numarası,Müşteri Adı,Vergi Dairesi,Sipariş Onay Tarihi,Kargolanma Tarihi,Maksimum Kargolama Tarihi,Kargo Ücreti,Depo,Komisyon Oranı,Komisyon
0,208594813238,Tamamlandı,02-Tem-2024 15:01,29-Haz-2024 15:34,601590236,004235-57857,004235-57857,"Paslanmaz Sarımsak Ezici, Sarımsak Ezeceği Met...",NaN,"79,00",...,2.580385e+09,emine çoban,Pendik,29-Haz-2024 15:36,01-Tem-2024 17:12,01-Tem-2024 23:59,Mağaza Öder (Şartlı Kargo),yeni ofis1,17.0,13.43
1,208594813238,Tamamlandı,02-Tem-2024 15:01,29-Haz-2024 15:34,551638890,001603,001603,Pikas Pati Mama Kabı 2 Kaseli Kedi Köpek Suluk...,Kahverengi,"159,00",...,2.580385e+09,emine çoban,Pendik,29-Haz-2024 15:36,01-Tem-2024 17:12,01-Tem-2024 23:59,Mağaza Öder (Şartlı Kargo),yeni ofis1,20.0,31.80
2,201928353236,Tamamlandı,01-Tem-2024 17:00,29-Haz-2024 00:04,598088935,003975-57558,003975-57558,Vardem Oyuncak Ib58330 Explorer Şişme Bot 200 ...,NaN,"550,00",...,NaN,Yeliz Uçgun Yaşar,NaN,29-Haz-2024 00:05,29-Haz-2024 13:13,01-Tem-2024 23:59,Mağaza Öder (Şartlı Kargo),yeni ofis1,16.0,88.00
3,205779413234,Tamamlandı,02-Tem-2024 12:02,29-Haz-2024 00:43,604303188,004314-58017,004314-58017,Küre Buz Kalıbı Kapaklı 33 Hazneli Yuvarlak Bu...,NaN,"60,00",...,NaN,Neslihan Yerlikaya,NaN,29-Haz-2024 00:45,29-Haz-2024 13:13,03-Tem-2024 23:59,Alıcı Tarafından Ödendi (Şartlı Kargo),yeni ofis1,17.0,10.20
4,202744353236,Tamamlandı,01-Tem-2024 13:00,29-Haz-2024 00:15,593821840,003667-57104,003667-57104,Termo Lüks Piknik Çantası Kırmızı Ekoseli 35 X...,NaN,"329,00",...,NaN,ilayda zerey,NaN,29-Haz-2024 00:18,29-Haz-2024 13:12,01-Tem-2024 23:59,Mağaza Öder (Şartlı Kargo),yeni ofis1,13.0,42.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,202914745232,Tamamlandı,04-Haz-2024 13:00,01-Haz-2024 23:11,590122139,003362-56683,003362-56683,"Kapatsiteli Styles Gıda Ve Bakliyat, Saklama K...",NaN,"430,00",...,NaN,Şenol Akarsu,NaN,01-Haz-2024 23:14,03-Haz-2024 13:32,03-Haz-2024 23:59,Mağaza Öder (Şartlı Kargo),yeni ofis1,17.0,73.10
655,209358545233,Tamamlandı,04-Haz-2024 15:01,02-Haz-2024 01:01,580699164,002583-55801,3644-55801,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",Beyaz,"149,00",...,NaN,NaN,NaN,02-Haz-2024 01:04,03-Haz-2024 13:35,03-Haz-2024 23:59,Alıcı Tarafından Ödendi (Şartlı Kargo),yeni ofis1,16.0,23.84
656,202317845238,Tamamlandı,04-Haz-2024 13:01,01-Haz-2024 20:23,580699164,002583-55801,3644-55801,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",Beyaz,"149,00",...,NaN,NaN,NaN,01-Haz-2024 20:26,03-Haz-2024 13:28,03-Haz-2024 23:59,Mağaza Öder (Şartlı Kargo),yeni ofis1,16.0,47.68
657,202145137230,Tamamlandı,05-Haz-2024 16:02,01-Haz-2024 16:26,569694341,002046-54748,002046-54748,Müller Klima Temizleme Köpük Sprey (Üstün Alma...,NaN,"144,49",...,4.740132e+09,güner salihi,AKSARAY,01-Haz-2024 16:27,03-Haz-2024 13:30,03-Haz-2024 23:59,Mağaza Öder (Şartlı Kargo),yeni ofis1,17.0,73.69


In [223]:
df_n11_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Sipariş Kodu               659 non-null    int64  
 1   Durum                      659 non-null    object 
 2   Sipariş Tamamlanma Tarihi  659 non-null    object 
 3   Ödeme Tarihi               659 non-null    object 
 4   Ürün Kodu                  659 non-null    int64  
 5   Mağaza Ürün Kodu           659 non-null    object 
 6   Stok Kodu                  659 non-null    object 
 7   Ürün Adı                   659 non-null    object 
 8   Seçenek                    285 non-null    object 
 9   Birim Fiyatı               659 non-null    object 
 10  Adet                       659 non-null    int64  
 11  Sipariş Tutarı             659 non-null    object 
 12  Mağaza İndirimi            659 non-null    int64  
 13  Kupon                      659 non-null    int64  

In [224]:
df_n11_raw.columns

Index(['Sipariş Kodu', 'Durum', 'Sipariş Tamamlanma Tarihi', 'Ödeme Tarihi',
       'Ürün Kodu', 'Mağaza Ürün Kodu', 'Stok Kodu', 'Ürün Adı', 'Seçenek',
       'Birim Fiyatı', 'Adet', 'Sipariş Tutarı', 'Mağaza İndirimi', 'Kupon',
       'Hizmet Bedeli (KDV Dahil)', 'Vade Farkı (KDV Dahil)',
       'Mağaza Fatura Tutarı', 'n11 Para Puanları', 'Tahsil Edilecek Tutar',
       'Kampanya Kodu', 'Teslimat Metodu', 'Kargo Şirketi', 'Takip Numarası',
       'Müşteri', 'Telefon Numarası', 'Cep Telefonu Numarası', 'E-posta',
       'Adres', 'İlçe', 'İl', 'Ad Soyadı / Firma Adı', 'Fatura Adresi',
       'İlçe.1', 'İl.1', 'Fatura Türü', 'TC Kimlik No', 'Vergi Numarası',
       'Müşteri Adı', 'Vergi Dairesi', 'Sipariş Onay Tarihi',
       'Kargolanma Tarihi', 'Maksimum Kargolama Tarihi', 'Kargo Ücreti',
       'Depo', 'Komisyon Oranı', 'Komisyon'],
      dtype='object')

In [225]:
df_n11_raw_select = df_n11_raw[["Ürün Kodu","Mağaza Ürün Kodu","Stok Kodu","Ürün Adı","Birim Fiyatı","Adet","Sipariş Tutarı","Mağaza Fatura Tutarı","n11 Para Puanları","Tahsil Edilecek Tutar","Komisyon Oranı","Komisyon"]]
df_n11_raw_select

,Ürün Kodu,Mağaza Ürün Kodu,Stok Kodu,Ürün Adı,Birim Fiyatı,Adet,Sipariş Tutarı,Mağaza Fatura Tutarı,n11 Para Puanları,Tahsil Edilecek Tutar,Komisyon Oranı,Komisyon
0,601590236,004235-57857,004235-57857,"Paslanmaz Sarımsak Ezici, Sarımsak Ezeceği Met...","79,00",1,79,79,0,79,17.0,13.43
1,551638890,001603,001603,Pikas Pati Mama Kabı 2 Kaseli Kedi Köpek Suluk...,"159,00",1,159,159,0,159,20.0,31.80
2,598088935,003975-57558,003975-57558,Vardem Oyuncak Ib58330 Explorer Şişme Bot 200 ...,"550,00",1,550,550,0,550,16.0,88.00
3,604303188,004314-58017,004314-58017,Küre Buz Kalıbı Kapaklı 33 Hazneli Yuvarlak Bu...,"60,00",1,60,60,"32,01","27,99",17.0,10.20
4,593821840,003667-57104,003667-57104,Termo Lüks Piknik Çantası Kırmızı Ekoseli 35 X...,"329,00",1,329,329,50,279,13.0,42.77
...,...,...,...,...,...,...,...,...,...,...,...,...
654,590122139,003362-56683,003362-56683,"Kapatsiteli Styles Gıda Ve Bakliyat, Saklama K...","430,00",1,430,430,70,360,17.0,73.10
655,580699164,002583-55801,3644-55801,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","149,00",1,149,149,"16,39","132,61",16.0,23.84
656,580699164,002583-55801,3644-55801,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","149,00",2,298,298,60,238,16.0,47.68
657,569694341,002046-54748,002046-54748,Müller Klima Temizleme Köpük Sprey (Üstün Alma...,"144,49",3,"433,47","433,47",70,"363,47",17.0,73.69


In [226]:
df_n11_raw_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ürün Kodu              659 non-null    int64  
 1   Mağaza Ürün Kodu       659 non-null    object 
 2   Stok Kodu              659 non-null    object 
 3   Ürün Adı               659 non-null    object 
 4   Birim Fiyatı           659 non-null    object 
 5   Adet                   659 non-null    int64  
 6   Sipariş Tutarı         659 non-null    object 
 7   Mağaza Fatura Tutarı   659 non-null    object 
 8   n11 Para Puanları      659 non-null    object 
 9   Tahsil Edilecek Tutar  659 non-null    object 
 10  Komisyon Oranı         659 non-null    float64
 11  Komisyon               659 non-null    float64
dtypes: float64(2), int64(2), object(8)
memory usage: 61.9+ KB


In [227]:
# df_n11_raw_select['Birim Fiyatı'] = df_n11_raw_select['Birim Fiyatı'].str.split(',').str[0]  # Because of , type numeric
# df_n11_raw_select

In [228]:
# df_n11_raw_select["Birim Fiyatı"] = df_n11_raw_select["Birim Fiyatı"].apply(pd.to_numeric)
# df_n11_raw_select

In [229]:
# df_n11_raw_select["Birim Fiyatı"] = df_n11_raw_select["Birim Fiyatı"].str.replace(".","", regex=False).str.replace(",",".", regex=False).astype(float)
# df_n11_raw_select

In [230]:
string_numeric_columns = ["Birim Fiyatı","Sipariş Tutarı","Mağaza Fatura Tutarı","n11 Para Puanları","Tahsil Edilecek Tutar"]

In [231]:
for column in string_numeric_columns:
    df_n11_raw_select[f"{column}"] = df_n11_raw_select[f"{column}"].apply(lambda x: convert_to_float(x))

C:\Users\user\AppData\Local\Temp\ipykernel_6748\4149499364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n11_raw_select[f"{column}"] = df_n11_raw_select[f"{column}"].apply(lambda x: convert_to_float(x))


In [232]:
df_n11_raw_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ürün Kodu              659 non-null    int64  
 1   Mağaza Ürün Kodu       659 non-null    object 
 2   Stok Kodu              659 non-null    object 
 3   Ürün Adı               659 non-null    object 
 4   Birim Fiyatı           659 non-null    float64
 5   Adet                   659 non-null    int64  
 6   Sipariş Tutarı         659 non-null    float64
 7   Mağaza Fatura Tutarı   659 non-null    float64
 8   n11 Para Puanları      659 non-null    float64
 9   Tahsil Edilecek Tutar  659 non-null    float64
 10  Komisyon Oranı         659 non-null    float64
 11  Komisyon               659 non-null    float64
dtypes: float64(7), int64(2), object(3)
memory usage: 61.9+ KB


In [233]:
df_n11_raw_select["Stok Kodu"].nunique()

92

In [234]:
df_n11_raw_select["Mağaza Ürün Kodu"].nunique()

92

In [235]:
df_n11_raw_select["Ürün Kodu"].nunique()

92

In [236]:
df_n11_raw_select["Ürün Adı"].nunique()

98

In [237]:
df_n11_raw_select["Birim Fiyatı"].nunique()

72

In [238]:
df_n11_raw_select.loc[:,'Model Kodu'] = df_n11_raw_select['Stok Kodu'].str.split('-').str[0]
df_n11_raw_select

C:\Users\user\AppData\Local\Temp\ipykernel_6748\3064590490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n11_raw_select.loc[:,'Model Kodu'] = df_n11_raw_select['Stok Kodu'].str.split('-').str[0]


,Ürün Kodu,Mağaza Ürün Kodu,Stok Kodu,Ürün Adı,Birim Fiyatı,Adet,Sipariş Tutarı,Mağaza Fatura Tutarı,n11 Para Puanları,Tahsil Edilecek Tutar,Komisyon Oranı,Komisyon,Model Kodu
0,601590236,004235-57857,004235-57857,"Paslanmaz Sarımsak Ezici, Sarımsak Ezeceği Met...",79.00,1,79.00,79.00,0.00,79.00,17.0,13.43,004235
1,551638890,001603,001603,Pikas Pati Mama Kabı 2 Kaseli Kedi Köpek Suluk...,159.00,1,159.00,159.00,0.00,159.00,20.0,31.80,001603
2,598088935,003975-57558,003975-57558,Vardem Oyuncak Ib58330 Explorer Şişme Bot 200 ...,550.00,1,550.00,550.00,0.00,550.00,16.0,88.00,003975
3,604303188,004314-58017,004314-58017,Küre Buz Kalıbı Kapaklı 33 Hazneli Yuvarlak Bu...,60.00,1,60.00,60.00,32.01,27.99,17.0,10.20,004314
4,593821840,003667-57104,003667-57104,Termo Lüks Piknik Çantası Kırmızı Ekoseli 35 X...,329.00,1,329.00,329.00,50.00,279.00,13.0,42.77,003667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,590122139,003362-56683,003362-56683,"Kapatsiteli Styles Gıda Ve Bakliyat, Saklama K...",430.00,1,430.00,430.00,70.00,360.00,17.0,73.10,003362
655,580699164,002583-55801,3644-55801,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",149.00,1,149.00,149.00,16.39,132.61,16.0,23.84,3644
656,580699164,002583-55801,3644-55801,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",149.00,2,298.00,298.00,60.00,238.00,16.0,47.68,3644
657,569694341,002046-54748,002046-54748,Müller Klima Temizleme Köpük Sprey (Üstün Alma...,144.49,3,433.47,433.47,70.00,363.47,17.0,73.69,002046


In [167]:
df_n11_raw_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ürün Kodu              659 non-null    int64  
 1   Mağaza Ürün Kodu       659 non-null    object 
 2   Stok Kodu              659 non-null    object 
 3   Ürün Adı               659 non-null    object 
 4   Birim Fiyatı           659 non-null    object 
 5   Adet                   659 non-null    int64  
 6   Sipariş Tutarı         659 non-null    object 
 7   Mağaza Fatura Tutarı   659 non-null    object 
 8   n11 Para Puanları      659 non-null    object 
 9   Tahsil Edilecek Tutar  659 non-null    object 
 10  Komisyon Oranı         659 non-null    float64
 11  Komisyon               659 non-null    float64
 12  Model Kodu             659 non-null    object 
dtypes: float64(2), int64(2), object(9)
memory usage: 67.1+ KB


In [239]:
df_n11_select_1 = df_n11_raw_select[["Model Kodu","Adet","Sipariş Tutarı","Mağaza Fatura Tutarı","n11 Para Puanları","Tahsil Edilecek Tutar"]]
df_n11_select_2 = df_n11_raw_select[["Model Kodu","Birim Fiyatı","Mağaza Ürün Kodu","Stok Kodu","Ürün Adı","Komisyon Oranı","Komisyon"]]

In [240]:
df_n11_select_11 = df_n11_select_1.groupby(["Model Kodu"]).sum().reset_index()
df_n11_select_11

,Model Kodu,Adet,Sipariş Tutarı,Mağaza Fatura Tutarı,n11 Para Puanları,Tahsil Edilecek Tutar
0,000699,2,300.00,300.00,60.75,239.25
1,001537,2,139.24,139.24,19.04,120.20
2,001601pktb,2,678.30,678.30,87.31,590.99
3,001601pktg,5,1695.75,1695.75,261.47,1450.98
4,001602,2,345.00,345.00,50.00,312.17
...,...,...,...,...,...,...
87,3648,14,2940.00,2940.00,359.73,2580.27
88,3724,2,499.98,499.98,43.10,456.88
89,3741,1,169.99,169.99,0.00,169.99
90,3777,1,604.99,604.99,66.55,538.44


In [241]:
df_n11_select_21 = df_n11_select_2.drop_duplicates(["Model Kodu"])
df_n11_select_21.reset_index(drop=True, inplace=True)
df_n11_select_21

,Model Kodu,Birim Fiyatı,Mağaza Ürün Kodu,Stok Kodu,Ürün Adı,Komisyon Oranı,Komisyon
0,004235,79.00,004235-57857,004235-57857,"Paslanmaz Sarımsak Ezici, Sarımsak Ezeceği Met...",17.0,13.43
1,001603,159.00,001603,001603,Pikas Pati Mama Kabı 2 Kaseli Kedi Köpek Suluk...,20.0,31.80
2,003975,550.00,003975-57558,003975-57558,Vardem Oyuncak Ib58330 Explorer Şişme Bot 200 ...,16.0,88.00
3,004314,60.00,004314-58017,004314-58017,Küre Buz Kalıbı Kapaklı 33 Hazneli Yuvarlak Bu...,17.0,10.20
4,003667,329.00,003667-57104,003667-57104,Termo Lüks Piknik Çantası Kırmızı Ekoseli 35 X...,13.0,42.77
...,...,...,...,...,...,...,...
87,3644,149.00,002583-55801,3644-55801,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",16.0,23.84
88,003549,99.99,003549-56957,003549-56957,5 Metre Usb Kumandalı Peri Led 8 Animasyonlu R...,21.0,21.00
89,003348,225.00,003348-56671,003348-56671,Bebelav Adalılar Kauçuk Termofor Kokusuz Sıcak...,16.0,36.00
90,001684,280.00,001684-54001,001684-54001,Quadro LK001 Temassız Ateş Ölçer,16.0,44.80


In [242]:
df_n11_select_12_21 = pd.merge(df_n11_select_11, df_n11_select_21, how="left", on="Model Kodu")
df_n11_select_12_21.drop_duplicates(inplace=True)
df_n11_select_12_21.reset_index(drop=True, inplace=True)
df_n11_select_12_21

,Model Kodu,Adet,Sipariş Tutarı,Mağaza Fatura Tutarı,n11 Para Puanları,Tahsil Edilecek Tutar,Birim Fiyatı,Mağaza Ürün Kodu,Stok Kodu,Ürün Adı,Komisyon Oranı,Komisyon
0,000699,2,300.00,300.00,60.75,239.25,150.00,000699-19976,000699-19976,Hobby Life Alaska Nofrost Saklama Kabı 185 Lit...,17.0,25.50
1,001537,2,139.24,139.24,19.04,120.20,69.62,001537-52013,001537-52013,Blue Focus 50 Farklı Hayvan Türkçe-İngilizce A...,18.0,12.53
2,001601pktb,2,678.30,678.30,87.31,590.99,339.15,001601pktb-53615,001601pktb-53615,Keçe Kedi ve Köpek Evi Yatağı-Mama Kabı 2 Kase...,20.0,67.83
3,001601pktg,5,1695.75,1695.75,261.47,1450.98,339.15,001601pktg-53616,001601pktg-53616,Alas Gri Keçe Kedi ve Köpek Evi Yatağı-Mama Ka...,20.0,67.83
4,001602,2,345.00,345.00,50.00,312.17,172.50,001602-53192,001602-53192,Oval Pati Desenli Ahşap İkili Çelik Kaseli Ked...,20.0,69.00
...,...,...,...,...,...,...,...,...,...,...,...,...
87,3648,14,2940.00,2940.00,359.73,2580.27,210.00,3648-55801,3648-55801,Portatif Katlanabilir Kamp Piknik Masası 001 S...,16.0,33.60
88,3724,2,499.98,499.98,43.10,456.88,249.99,002672-55888,3724-55888,Pikas İsme Özel Yıkanabilir Peti Desenli Kedi ...,20.0,50.00
89,3741,1,169.99,169.99,0.00,169.99,169.99,3741-55974,3741-55974,"Brillahome Kaymaz Taban, Tekli Post Peluş Bany...",18.5,31.45
90,3777,1,604.99,604.99,66.55,538.44,604.99,3777-56086,3777-56086,Brillahome Dijital Baskı Yıkanabilir Pamuk Tab...,18.5,111.92


In [243]:
df_n11_select_12_21[df_n11_select_12_21["Stok Kodu"] == '002066-54768']

,Model Kodu,Adet,Sipariş Tutarı,Mağaza Fatura Tutarı,n11 Para Puanları,Tahsil Edilecek Tutar,Birim Fiyatı,Mağaza Ürün Kodu,Stok Kodu,Ürün Adı,Komisyon Oranı,Komisyon
10,002066,9,816.75,816.75,117.28,699.47,90.75,002066-54768,002066-54768,Müller RADYATÖR ÇATLAK GİDERİCİ (ÜSTÜN ALMAN ...,13.0,11.8


In [244]:
df_n11_select_12_21_price = pd.merge(df_n11_select_12_21, df_price, how="left", on="Model Kodu")
df_n11_select_12_21_price

,Model Kodu,Adet,Sipariş Tutarı,Mağaza Fatura Tutarı,n11 Para Puanları,Tahsil Edilecek Tutar,Birim Fiyatı,Mağaza Ürün Kodu,Stok Kodu,Ürün Adı_x,Komisyon Oranı,Komisyon,Ürün Adı_y,Gelişi (KDV DAHİL)
0,000699,2,300.00,300.00,60.75,239.25,150.00,000699-19976,000699-19976,Hobby Life Alaska Nofrost Saklama Kabı 185 Lit...,17.0,25.50,NaN,NaN
1,001537,2,139.24,139.24,19.04,120.20,69.62,001537-52013,001537-52013,Blue Focus 50 Farklı Hayvan Türkçe-İngilizce A...,18.0,12.53,NaN,NaN
2,001601pktb,2,678.30,678.30,87.31,590.99,339.15,001601pktb-53615,001601pktb-53615,Keçe Kedi ve Köpek Evi Yatağı-Mama Kabı 2 Kase...,20.0,67.83,NaN,NaN
3,001601pktg,5,1695.75,1695.75,261.47,1450.98,339.15,001601pktg-53616,001601pktg-53616,Alas Gri Keçe Kedi ve Köpek Evi Yatağı-Mama Ka...,20.0,67.83,NaN,NaN
4,001602,2,345.00,345.00,50.00,312.17,172.50,001602-53192,001602-53192,Oval Pati Desenli Ahşap İkili Çelik Kaseli Ked...,20.0,69.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,3648,14,2940.00,2940.00,359.73,2580.27,210.00,3648-55801,3648-55801,Portatif Katlanabilir Kamp Piknik Masası 001 S...,16.0,33.60,NaN,NaN
88,3724,2,499.98,499.98,43.10,456.88,249.99,002672-55888,3724-55888,Pikas İsme Özel Yıkanabilir Peti Desenli Kedi ...,20.0,50.00,NaN,NaN
89,3741,1,169.99,169.99,0.00,169.99,169.99,3741-55974,3741-55974,"Brillahome Kaymaz Taban, Tekli Post Peluş Bany...",18.5,31.45,NaN,NaN
90,3777,1,604.99,604.99,66.55,538.44,604.99,3777-56086,3777-56086,Brillahome Dijital Baskı Yıkanabilir Pamuk Tab...,18.5,111.92,NaN,NaN


In [245]:
df_n11_select_12_21_price.columns

Index(['Model Kodu', 'Adet', 'Sipariş Tutarı', 'Mağaza Fatura Tutarı',
       'n11 Para Puanları', 'Tahsil Edilecek Tutar', 'Birim Fiyatı',
       'Mağaza Ürün Kodu', 'Stok Kodu', 'Ürün Adı_x', 'Komisyon Oranı',
       'Komisyon', 'Ürün Adı_y', 'Gelişi (KDV DAHİL)'],
      dtype='object')

In [246]:
df_n11_select_12_21_price = df_n11_select_12_21_price[['Model Kodu','Stok Kodu','Mağaza Ürün Kodu','Ürün Adı_x','Ürün Adı_y','Adet','Birim Fiyatı','Sipariş Tutarı','Gelişi (KDV DAHİL)','Mağaza Fatura Tutarı','Tahsil Edilecek Tutar','n11 Para Puanları','Komisyon Oranı','Komisyon']]
df_n11_select_12_21_price

,Model Kodu,Stok Kodu,Mağaza Ürün Kodu,Ürün Adı_x,Ürün Adı_y,Adet,Birim Fiyatı,Sipariş Tutarı,Gelişi (KDV DAHİL),Mağaza Fatura Tutarı,Tahsil Edilecek Tutar,n11 Para Puanları,Komisyon Oranı,Komisyon
0,000699,000699-19976,000699-19976,Hobby Life Alaska Nofrost Saklama Kabı 185 Lit...,NaN,2,150.00,300.00,NaN,300.00,239.25,60.75,17.0,25.50
1,001537,001537-52013,001537-52013,Blue Focus 50 Farklı Hayvan Türkçe-İngilizce A...,NaN,2,69.62,139.24,NaN,139.24,120.20,19.04,18.0,12.53
2,001601pktb,001601pktb-53615,001601pktb-53615,Keçe Kedi ve Köpek Evi Yatağı-Mama Kabı 2 Kase...,NaN,2,339.15,678.30,NaN,678.30,590.99,87.31,20.0,67.83
3,001601pktg,001601pktg-53616,001601pktg-53616,Alas Gri Keçe Kedi ve Köpek Evi Yatağı-Mama Ka...,NaN,5,339.15,1695.75,NaN,1695.75,1450.98,261.47,20.0,67.83
4,001602,001602-53192,001602-53192,Oval Pati Desenli Ahşap İkili Çelik Kaseli Ked...,NaN,2,172.50,345.00,NaN,345.00,312.17,50.00,20.0,69.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,3648,3648-55801,3648-55801,Portatif Katlanabilir Kamp Piknik Masası 001 S...,NaN,14,210.00,2940.00,NaN,2940.00,2580.27,359.73,16.0,33.60
88,3724,3724-55888,002672-55888,Pikas İsme Özel Yıkanabilir Peti Desenli Kedi ...,NaN,2,249.99,499.98,NaN,499.98,456.88,43.10,20.0,50.00
89,3741,3741-55974,3741-55974,"Brillahome Kaymaz Taban, Tekli Post Peluş Bany...",NaN,1,169.99,169.99,NaN,169.99,169.99,0.00,18.5,31.45
90,3777,3777-56086,3777-56086,Brillahome Dijital Baskı Yıkanabilir Pamuk Tab...,NaN,1,604.99,604.99,NaN,604.99,538.44,66.55,18.5,111.92


In [ ]:
# df_n11_select_12_21_price.to_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Result\Test_Result\Haziran N11.xlsx", index=False)

#### Hepsiburada

In [ ]:
df_hepsi_raw = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Satış Pazarlama\HEPSİ BURADA SATIŞ RAPORLARI\nisan 2024 hepsi burada.xlsx")
df_hepsi_raw

In [ ]:
df_hepsi_raw = df_hepsi_raw.rename(columns={"Urun Adi":"Ürün Adı"})
df_hepsi_raw

In [ ]:
df_hepsi_raw_price = pd.merge(df_hepsi_raw, df_price, how="left", on="Ürün Adı")
df_hepsi_raw_price

In [ ]:
df_hepsi_raw_price.to_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Result\Test_Result\Nisan 2024 Hepsi.xlsx", index=False)

#### Trendyol

In [ ]:
df_trend_raw = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Satış Pazarlama\TRENDYOL SATIŞ RAPORLARI\haziran 2024 -satış-raporu-.xlsx")
df_trend_raw

In [ ]:
df_trend_raw.info()

In [ ]:
df_trend_raw.columns

In [ ]:
df_trend_select_31 = df_trend_raw.drop("Barkod", axis=1)
df_trend_select_31

In [ ]:
df_trend_select_31_price = pd.merge(df_trend_select_31, df_price, how="left", on="Model Kodu")
df_trend_select_31_price

In [ ]:
df_trend_select_31_price.to_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Result\Test_Result\Haziran 2024 Trendyol.xlsx", index=False)

#### Temp

In [ ]:
def select_code(kode):
    pattern = re.compile(r"^00(.*)")
    try:
        match = pattern.match(kode)
        result = match.group(1)
    except:
        result = kode
    return result

In [ ]:
kode = "1601pktb"

In [ ]:
select_code(kode)

In [ ]:
df_n11_select_12_21["Model Kodu"] = df_n11_select_12_21["Model Kodu"].apply(lambda kode: select_code(kode))
df_price["Model Kodu"] = df_price["Model Kodu"].apply(lambda kode: select_code(kode))

In [ ]:
df_temp = pd.merge(df_n11_select_12_21, df_price, how="left", on="Model Kodu")
df_temp

In [ ]:
df_temp.to_excel("Haziran n11 2.xlsx", index=False)

In [ ]:
# df_june_select = df_june_select.groupby(["Stok Kodu","Mağaza Ürün Kodu","Ürün Kodu","Ürün Adı"]).sum().reset_index()
# df_june_select["Birim Fiyatı"] = df_june_select["Birim Fiyatı"]/df_june_select["Adet"]
# df_june_select

In [ ]:
set_a = set(df_n11_select_12_21["Model Kodu"])
set_b = set(df_price["Model Kodu"])

In [ ]:
set_a.difference(set_b)

#### Ürün Kod Birleştirme

In [ ]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text = text.strip()  # rstrip lstrip
    return text

In [ ]:
text = " Ali top oyna"

In [ ]:
whitespace_del(text)

In [ ]:
df1 = pd.read_excel(r"C:\Users\user\Desktop\Hepsi Burada.xlsx")
df1

In [ ]:
df1["Ürün Adı"] = df1["Ürün Adı"].apply(lambda x: whitespace_del(x))
df1

In [ ]:
df1 = df1.drop_duplicates()
df1.reset_index(drop=True, inplace=True)
df1

In [ ]:
df2 = pd.read_excel(r"C:\Users\user\Desktop\Trendyol.xlsx")
df2

In [ ]:
df2["Ürün Adı"] = df2["Ürün Adı"].apply(lambda x: whitespace_del(x))
df2

In [ ]:
df2 = df2.drop_duplicates()
df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
df3 = pd.read_excel(r"C:\Users\user\Desktop\N11.xlsx")
df3

In [ ]:
df3["Ürün Adı"] = df3["Ürün Adı"].apply(lambda x: whitespace_del(x))
df3

In [ ]:
df3 = df3.drop_duplicates()
df3.reset_index(drop=True, inplace=True)
df3

In [ ]:
df_var = pd.merge(df1,df2,how="outer", on="Ürün Adı")
df_var

In [ ]:
df_var2 = pd.merge(df_var,df3,how="outer", on="Ürün Adı")
df_var2

In [ ]:
df_var2 = df_var2.drop_duplicates()
df_var2

In [ ]:
df_var2.sort_values(by="Ürün Adı", ascending=True)

In [ ]:
df_var2.to_excel(r"C:\Users\user\Desktop\Ürün Kod Listesi.xlsx", index=False)